In [ ]:
######################

In [1]:
import pandas as pd
from openai import OpenAI, OpenAIError
import yaml
import time

llm_iter_counter = 0


In [2]:
# Подгрузка ключей для openai
with open (r'C:\Users\lutsevich\Desktop\py\keys-openrouter-free-deepseek.yml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)
api_keys = [item['key'] for item in config.get('api_keys', []) if item.get('env') == 'org-find']

In [3]:
# Импорт базы
df = pd.read_excel(r"V:\00_Базы\04_Базы регистрации\База ГТН-РБ\база 2019-6 мес. 2025 ГТН.xlsx", sheet_name='основной массив', engine='openpyxl')

# Выходной датафрейм
df_exit = pd.DataFrame(columns=['title', 'location', 'district', 'answer'])

c:\Users\lutsevich\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q6031 is marked as a date but the serial value 3201101103 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lutsevich\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q6052 is marked as a date but the serial value 21007524 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lutsevich\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q6103 is marked as a date but the serial value 19000185 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lutsevich\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell Q6172 is marked as a date but the serial value 101810021080 i

In [4]:
# Чистка df
# 1. Убираем дубликаты по наименованию
df_m = df[['Дата регистрации','Эксплуатирующая организация','Вид деятельности эксплуатирующей организации','Область','Район']].drop_duplicates(subset='Эксплуатирующая организация')
# 2. Оставляем только те, у которых нет сферы деятельности
df_m = df_m[df_m['Вид деятельности эксплуатирующей организации'].isna()]
# 3. Заменяем кавычки
df_m['Эксплуатирующая организация'] = df_m['Эксплуатирующая организация'].str.replace('"', "'")
df_m.describe() # 3906 count

,Дата регистрации
count,3906
mean,2023-02-18 20:04:25.437787904
min,2019-01-03 00:00:00
25%,2022-01-27 00:00:00
50%,2023-05-31 00:00:00
75%,2024-10-01 00:00:00
max,2025-06-30 00:00:00


In [5]:
# Функция запроса к ллм-ке
def run_llm(api_keys, title, location, district):
  global llm_iter_counter
  
  # Цикл ключей
  api_key = api_keys[llm_iter_counter % len(api_keys)]
  # Промт вар 2 для сборки исходные + выход
  messages=[
  {
      "role": "system",
      "content": 
          "Привет! Я даю тебе юридическое наименование и локацию предприятия из Беларуси, ты возвращаешь мне текст со сферой деятельности этого предприятия"
          "Если не достигается высокая точность, то присваивай н.д."
          "Примеры:"
          "Вход - Имя: ГОСУДАРСТВЕННОЕ ПРЕДПРИЯТИЕ 'ЗЕЛЬВЕНСКОЕ ПМС'; Область: Гродненская; Район: Зельвянский Выход - Мелиоративные предприятия'"
          "Вход - Имя: БОРИСОВСКИЙ ОПЫТНЫЙ ЛЕСХОЗ; Область: Минская; Район: Борисовский Выход - Лесхозы"
          "Вход - Имя: СЗАО 'РИНДИБЕЛ'; Область: Гродненская; Район: Волковысский Выход - Деревообработка"
          "Вход - КСУП 'БЫВАЛЬКИ'; Область: Гомельская; Район: Лоевский Выход - Сельское хозяйство"
          "Вход - Вход - ФИЛИАЛ ДЭУ-63 РУП 'МИНСКАВТОДОР-ЦЕНТР'; Область: Минская; Район: Вилейский Выход - Деятельность по эксплуатации автомобильных дорог"
  },
  {
      "role": 'user', 'content': f'Вход - Имя: {title}; Область: {location}; Район: {district}'
  } 
  ]
  # Выполнение
  client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
  )

  completion = client.chat.completions.create(
    extra_headers={
      "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
      "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
    },
    extra_body={},
    model="deepseek/deepseek-chat-v3.1:free",
    messages=messages
  )
  return completion.choices[0].message.content

In [ ]:
# Подстановка значений по индексу
for i, row in df_m.iterrows():
    if i > max(df_exit.index): # 9620 (missing 2816 - 6514)
        title = row['Эксплуатирующая организация'] 
        location = row['Область']
        district = row['Район']
        try:
            answer = run_llm(api_keys, title, location, district)
            llm_iter_counter +=1
            print(f'{i}: Имя: {title}; Область: {location}; Район: {district}; Answer: {answer}')
            df_exit.loc[i] = [title, location, district, answer]
            time.sleep(5)
        except OpenAIError as e:
            print(f"Error {e.body['code']} at index {i}: '{e.body['message']}'")
            break

9618: Имя: ООО 'САНЛЕС'; Область: Минская; Район: Слуцкий; Answer: н.д.
9620: Имя: ЗАО 'ГОМЕЛЬСПЕЦСТРОЙ'; Область: Гомельская; Район: г. Гомель; Answer: Строительство
Error 429 at index 9621: 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day'


In [11]:
llm_iter_counter +=1

In [13]:
path = r'V:\00_Базы\04_Базы регистрации\orgs2.xlsx'
df_exit.to_excel(path)